# Classify Hymenoptera species

In [1]:
##############################################################################
#                                                                            #
#                ▄▄████████████▄ ▐                                           #
#             ▄███▀▀▀▀▀▀▀████████▐                                           #
#         ▄▀▀▀▄▄▄▄███████▄▄▄▄▀▀▀█▐                                           #
#      ▄▛ ▟█████████████████████▄▐                                           #
#    ▄█▛ ▟█         ▀▚   █████   ▐                                           #
#   ███ ██▌   ▄▄▄▖   ▐  ▐████▌   ▐    ████████ ████████  ▟████▙   ██     ██  #
#  ████ ██▌   ███▛   ▐  ▐████▌   ▐       ██    ██       ▟█▛  ▜█▙  ██     ██  #
#  ████ ▀█           ▞  █████    ▐       ██    ███████  ██        █████████  #
#  █████ █   ▄▄▄▄▄▄▄▞   ▜██▛    ▐▐       ██    ██       ▜█▙  ▟█▛  ██     ██  #
#   █████    ██████            ▐█▐       ██    ████████  ▜████▛   ██     ██  #
#    █████  ███████▙          ▟██▐                                           #
#     ▀████▙▀▀▀█████████████▀▀▀▄▄▐                                           #
#       ▀██████▄▄▄▄▀▀▀▀▀▄▄▄▄█████▐                                           #
#          ▀███████████████████▀ ▐                                           #
#             ▀▀███████████▀▀    ▐                                           #
#                                                                            #
##############################################################################

In [2]:
# import the necessary packages
import numpy as np
import cv2
import os
import torch
import torchvision
import yaml
import matplotlib.pyplot as plt

# Constants

In [3]:
DATA_PATH = "./dataset/Hymenoptera.v2i.yolov5pytorch"
MODEL_PATH = "./yolov5"

# Train the Yolo model

Avant d'executer la commande, il faut clonner le projet yolov5 sur votre machine:   
```bash
git clone git@github.com:ultralytics/yolov5.git
cd yolov5
python train.py --img 640 --batch 16 --epochs 3 --data ../dataset/Hymenoptera.v2i.yolov5pytorch/data.yaml --weights yolov5m.pt --device cpu
cd ..
tensorboard --logdir yolov5/runs/train
cd yolov5
python export.py --weights ./runs/train/exp15/weights/best.pt --include tflite
```

# Load model

```bash
cd yolov5
!python detect.py --weights ./runs/train/exp15/weights/best.pt --source ../image/abeilles.jpg --output ../image/abeilles_detected.jpg
```

In [4]:
import tensorflow as tf

# define EXT_TRAIN
EXT_TRAIN = "15"

In [5]:
model = tf.keras.models.load_model(MODEL_PATH + "/runs/train/exp"+EXT_TRAIN+"/weights/best_saved_model")

2022-02-10 15:16:29.431563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-10 15:16:29.452288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sylvain/.pyenv/versions/3.8.9/envs/UseCase/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda-11.6/lib64:
2022-02-10 15:16:29.452300: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-02-10 15:16:29.45259

In [6]:
# get labels in data.yaml
with open(DATA_PATH + "/data.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
    labels = data["names"]
labels

['Frelon Europ-en',
 'anthophila',
 'vespa orientalis',
 'vespa velutina',
 'vespoidea']

# Testing with a new image

In [7]:
# load the image from disk
image = cv2.imread('./image/abeilles.jpg')
# preprocess the image for classification
image_preprocessed = cv2.resize(image, (640, 640))
image_preprocessed = image_preprocessed.astype("float32") / 255.0

In [52]:
output = model.predict(np.expand_dims(image_preprocessed, axis=0))[0]
# get best found class
results = []
for i in range(len(output)):
    if output[i][4] > 0.5:
        results.append(output[i])


In [51]:
for result in results:
    indice = np.argmax(result[5:10])
    label = "{}: {:.2f}%".format(labels[4-indice], result[indice+5] * 100)
    print(label)

In [10]:
results

[array([0.17813344, 0.55643225, 0.3370436 , 0.62242335, 0.6598201 ,
        0.00329047, 0.00472698, 0.00498188, 0.750872  , 0.4347657 ],
       dtype=float32),
 array([0.18071966, 0.568867  , 0.339387  , 0.65152115, 0.8343003 ,
        0.00375861, 0.00546089, 0.00482097, 0.67052007, 0.5111882 ],
       dtype=float32),
 array([0.18426982, 0.5712788 , 0.3479124 , 0.64826554, 0.529524  ,
        0.005263  , 0.01008776, 0.00778481, 0.71018445, 0.30698842],
       dtype=float32),
 array([0.6941573 , 0.92268103, 0.44461986, 0.15436   , 0.5011538 ,
        0.01192474, 0.00619721, 0.00391212, 0.7567251 , 0.44517294],
       dtype=float32),
 array([0.1786963 , 0.5566897 , 0.344885  , 0.6235915 , 0.6539835 ,
        0.00293434, 0.00529262, 0.00535089, 0.7144984 , 0.42759818],
       dtype=float32),
 array([0.17970994, 0.5684565 , 0.35184672, 0.6479449 , 0.833331  ,
        0.00319496, 0.00536299, 0.005032  , 0.63425493, 0.50386536],
       dtype=float32),
 array([0.18437362, 0.5717573 , 0.355556

In [30]:
image.shape

(500, 885, 3)

In [43]:
final_image = image.copy()

for result in results:
    # draw the box on the image
    x, y, w, h = result[:4]
    print(x, y, w, h)
    startX = int((x - w / 2) * image.shape[0])
    startY = int((y - h / 2) * image.shape[1])
    endX = int((x + w / 2) * image.shape[0])
    endY = int((y + h / 2) * image.shape[1])
    print(startX, startY, endX, endY)

    indice = np.argmax(result[5:])
    label = "{}: {:.2f}%".format(labels[indice], result[indice+5] * 100)

    final_image = cv2.rectangle(final_image, (startX, startY), (endX, endY), (0, 255, 0), 2)
    # draw the predicted label on the image
    final_image = cv2.putText(final_image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

# show the output image
cv2.imshow("Image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.17813344 0.55643225 0.3370436 0.62242335
4 217 173 767
0.18071966 0.568867 0.339387 0.65152115
5 215 175 791
0.18426982 0.5712788 0.3479124 0.64826554
5 218 179 792
0.6941573 0.92268103 0.44461986 0.15436
235 748 458 884
0.1786963 0.5566897 0.344885 0.6235915
3 216 175 768
0.17970994 0.5684565 0.35184672 0.6479449
1 216 177 789
0.18437362 0.5717573 0.3555564 0.64387167
3 221 181 790


## Metrics

In [12]:
# print([a.shape for a in output])

In [13]:
# y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
# metrics.confusion_matrix(y_true, y_pred, labels=labels)